# Résolution d'un problème d'optimisation de type sac-à-dos multidimentionnel (d-Kp) par l'algorithme du simplex sous Python

##Résolution avec le code de l'algorithme

In [1]:
# fonction "objectif": maximise 200*x_1 + 150*x_2
Z = [200, 150]
# Coefficients des contraintes de disponibilités
A = [
        [1, 0],
        [0, 1],
        [1, 1],
        [2, 1]
    ]
# les disponiblilités:
b = [400, 700, 800, 1000]

In [2]:
#Initialisation
#Ajout des "slack" c'est à dire des variables d'écart 
# Ici on ajoute 4 "slack" associés aux 4 disponibilités

Z = [200, 150, 0, 0, 0, 0]

A = [
        [1, 0,  1, 0, 0, 0],
        [0, 1,  0, 1, 0, 0],
        [1, 1,  0, 0, 1, 0],
        [2, 1,  0, 0, 0, 1]
    ]

b = [400, 700, 800, 1000]

In [3]:
#définition d'une fonction pour le tableau initial
def initialTableau(Z, A, b):
   tableau = [row[:] + [x] for row, x in zip(A, b)]
   tableau.append(Z[:] + [0])
   return tableau

In [4]:
#voir notre tableau initial
initialTableau(Z, A, b)

[[1, 0, 1, 0, 0, 0, 400],
 [0, 1, 0, 1, 0, 0, 700],
 [1, 1, 0, 0, 1, 0, 800],
 [2, 1, 0, 0, 0, 1, 1000],
 [200, 150, 0, 0, 0, 0, 0]]

In [5]:
#définition d'une fonction pour tester la perfectibilité d'un tableau (d'une solution)
def canImprove(tableau):
   lastRow = tableau[-1]
   return any(x > 0 for x in lastRow[:-1])

In [7]:
#définition d'une fonction pour trouver un pivot
def findPivotIndex(tableau):
   # pick first nonzero index of the last row (colonne pivot)
   column = [i for i,x in enumerate(tableau[-1][:-1]) if x > 0][0]
   quotients = [(i, r[-1] / r[column]) for i,r in enumerate(tableau[:-1]) if r[column] > 0]
 
   # pick row index minimizing the quotient (ligne pivot)
   row = min(quotients, key=lambda x: x[1])[0]
   return row, column

In [12]:
#définition d'une fonction pour l'itération (pivotage)
def pivotAbout(tableau, pivot):
   i,j = pivot
 
   pivotDenom = tableau[i][j]
   tableau[i] = [x / pivotDenom for x in tableau[i]]
 
   for k,row in enumerate(tableau):
      if k != i:
         pivotRowMultiple = [y * tableau[k][j] for y in tableau[i]]
         tableau[k] = [x - y for x,y in zip(tableau[k], pivotRowMultiple)]

In [13]:
#définition d'une fonction simplex composée de toutes les fonctions précédentes
def simplex(Z, A, b):
   tableau = initialTableau(Z, A, b)
 
   while canImprove(tableau):
      pivot = findPivotIndex(tableau)
      pivotAbout(tableau, pivot)
 
   return tableau

In [14]:
#voir notre tableau final
finalTableau = simplex(Z, A, b)
finalTableau

[[1.0, 0.0, 0.0, 0.0, -1.0, 1.0, 200.0],
 [0.0, 0.0, 0.0, 1.0, -2.0, 1.0, 100.0],
 [0.0, 0.0, 1.0, 0.0, 1.0, -1.0, 200.0],
 [0.0, 1.0, 0.0, 0.0, 2.0, -1.0, 600.0],
 [0.0, 0.0, 0.0, 0.0, -100.0, -50.0, -130000.0]]

In [27]:
#Extraction des résultats

#solution primale
def primalSolution(tableau):
  res = {"x_1" : tableau[0][-1], "x_2" : tableau[3][-1], "M1" : tableau[2][-1],"M2" : tableau[1][-1]}
  return res

#valeur de l'objectif
def objectiveValue(tableau):
   return -(tableau[-1][-1])

#shadow price (prix de référence des ressources saturées ou épuisées)
def shadowPrice(tableau):
  shadowPrice = {"P_M3" : -(tableau[-1][-3]), "P_M4" : -(tableau[-1][-2])}
  return shadowPrice

In [24]:
primalSolution(finalTableau)

{'M1': 200.0, 'M2': 100.0, 'x_1': 200.0, 'x_2': 600.0}

In [25]:
objectiveValue(finalTableau)

130000.0

In [28]:
shadowPrice(finalTableau)

{'P_M3': 100.0, 'P_M4': 50.0}

**Interpretation des resultats**

Le plan de production maximisant la vente est celui dans lequel les nombres de produits P1 et P2 fabriqués sont respectivement de 200 et 600 ; 
ce qui permet d'obtenir un chiffre d'affaire de 130000.

A ce stade de production, les disponibilités des machines M3 et M4 sont saturées, par contre les machines M1 et M2 ont respectivement 
des disponibilités restantes de 200 et 100.

Les machines saturées M3 et M4 ont pour prix de référence respectifs de 100 et 50. Autrement dit : une unité supplémentaire de disponibilité en M3 permettrait d'accroître le chiffre d'affaire de 200 unités alors qu'une unité supplémentaire de disponibilité en M4 permettrait d'accroître le chiffre d'affaire de 50 unités.



##Résolution avec la fonction linprog(methode = simplex) du module scipy.optimize

In [ ]:
# import de la fonction linprog (method="simplex") du module scipy.optimize
from scipy.optimize import linprog

In [ ]:
# Traduisons la formalisation du problème sous forme de dictionaire à entrer dans la fonction linprog

# fonction "objectif": maximiser 200*x_1 + 150*x_2  
#ce qui revient à : minimiser -200*x_1 -150*x_2
# => Sous les contraintes
# x1 ≤ 400 		          (contrainte liée à la disponibilité de la machine M1)
#	x2 ≤ 700              (contrainte liée à la disponibilité de la machine M2)
#	x1 + x2 ≤  800        (contrainte liée à la disponibilité de la machine M3)
#2x1 + x2 ≤ 1000        (contrainte liée à la disponibilité de la machine M4)
#x1 ; x2 ≥ 0            (contrainte liée à la positivité des quantités)


probleme = {
    # fonction "objectif": minimize -200*x_1 + -150*x_2
    "Z": [-200, -150],
    # Coefficients des contraintes de disponibilités
    "A_ub": [
        [1, 0],
        [0, 1],
        [1, 1],
        [2, 1],
    ],
    # les disponiblilités:
    "b_ub": [400, 700, 800, 1000],
    # En fin la contrainte de positivité des quantités: 0 <= x_i <= +oo 
    "bounds": (0, None),
}

In [ ]:
#introduction des valeurs du dictionaire "probleme" dans la fonction linprog(methode="simplex")
linprog(
        probleme["Z"],
        A_ub=probleme["A_ub"],
        b_ub=probleme["b_ub"],
        bounds=probleme["bounds"],
        method="simplex")

     con: array([], dtype=float64)
     fun: -130000.0
 message: 'Optimization terminated successfully.'
     nit: 5
   slack: array([200., 100.,   0.,   0.])
  status: 0
 success: True
       x: array([200., 600.])

détails des résultats

In [ ]:
# définissons d'abord une fonction composée de la fonction linprog afin de simplifier l'entrée du dictionnaire dans linprog 

def linprog_wrapper(problem, **kwargs):
    result = linprog(
        probleme["Z"],
        A_ub=probleme["A_ub"],
        b_ub=probleme["b_ub"],
        bounds=probleme["bounds"],
        method="simplex",
        **kwargs
    )
    return result

In [ ]:
#Ajoutons une fonction callback qui va nous permettre de présenter de manière plus détaillée les résultats
def dummy_callback(r):
    print(f"\n- Itération #{r['nit']}, phase {r['phase']} :")
    fun = r['fun']
    print(f"    Valeur objectif    = {fun}")
    slack = r['slack']
    print(f"    Variables d'écart  = {slack}")
    x = r['x']
    print(f"    Variables objectif = {x}")

In [ ]:
#exécution
linprog_wrapper(probleme, callback=dummy_callback)


- Itération #0, phase 1 :
    Valeur objectif    = 0.0
    Variables d'écart  = [ 400.  700.  800. 1000.]
    Variables objectif = [0. 0.]

- Itération #1, phase 1 :
    Valeur objectif    = -80000.0
    Variables d'écart  = [  0. 700. 400. 200.]
    Variables objectif = [400.   0.]

- Itération #2, phase 1 :
    Valeur objectif    = -110000.0
    Variables d'écart  = [  0. 500. 200.   0.]
    Variables objectif = [400. 200.]

- Itération #3, phase 1 :
    Valeur objectif    = -130000.0
    Variables d'écart  = [200. 100.   0.   0.]
    Variables objectif = [200. 600.]

- Itération #4, phase 1 :
    Valeur objectif    = -125000.0
    Variables d'écart  = [300.   0.   0. 100.]
    Variables objectif = [100. 700.]

- Itération #4, phase 2 :
    Valeur objectif    = -125000.0
    Variables d'écart  = [300.   0.   0. 100.]
    Variables objectif = [100. 700.]

- Itération #5, phase 2 :
    Valeur objectif    = -130000.0
    Variables d'écart  = [200. 100.   0.   0.]
    Variables objectif

     con: array([], dtype=float64)
     fun: -130000.0
 message: 'Optimization terminated successfully.'
     nit: 5
   slack: array([200., 100.,   0.,   0.])
  status: 0
 success: True
       x: array([200., 600.])

La fonction linprog(methode = simplex) du module scipy.optimize permet d'obtenir les mêmes résultats au bout de 5 itérations.